In [1]:
!pip install langchain langgraph langsmith langchain-groq langchain_community pypdf chromadb

INFO: pip is looking at multiple versions of langchain-groq to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 8.6 MB/s eta 0:00:00


In [2]:
import os
import uuid
from typing import TypedDict, Annotated, List, Dict, Any, Optional

# LangGraph / LangChain
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_groq import ChatGroq

from pypdf import PdfReader

# Vector DB (Chroma)
import chromadb
from chromadb.utils import embedding_functions
from chromadb.config import Settings

In [5]:
# Model

groq_api_key = "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"
LLM_MODEL_NAME = "openai/gpt-oss-20b"

llm = ChatGroq(model=LLM_MODEL_NAME, groq_api_key=groq_api_key)


# Vector DB Setup
chroma_client = chromadb.PersistentClient(
    path="./chroma_db",
    settings=Settings(anonymized_telemetry=False)
)
embedding_function = embedding_functions.DefaultEmbeddingFunction()

# Long-term memory collection (persistent across sessions)
LT_COLLECTION_NAME = "ra_longterm_insights"
try:
    lt_collection = chroma_client.get_collection(
        name=LT_COLLECTION_NAME,
        embedding_function=embedding_function
    )
except Exception:
    lt_collection = chroma_client.create_collection(
        name=LT_COLLECTION_NAME,
        embedding_function=embedding_function
    )


In [6]:
# Utilities
def extract_text_from_pdf(pdf_path: str) -> str:
    """Extract text from a PDF."""
    reader = PdfReader(pdf_path)
    texts = []
    for page in reader.pages:
        try:
            texts.append(page.extract_text() or "")
        except Exception:
            texts.append("")
    return "\n".join(texts).strip()

def simple_chunk_text(text: str, chunk_size: int = 1200, chunk_overlap: int = 200) -> List[str]:
    """Paragraph-first chunker with light sentence fallback and overlap."""
    if not text:
        return []
    paras = [p.strip() for p in text.split("\n\n") if p.strip()]
    chunks, buf, cur_len = [], [], 0

    def _flush_with_overlap():
        nonlocal buf, cur_len
        if buf:
            chunks.append(" ".join(buf).strip())
            if chunk_overlap > 0 and chunks[-1]:
                overlap_tokens = chunks[-1].split()[-(chunk_overlap // 6):]
                buf = [" ".join(overlap_tokens)]
                cur_len = len(buf[0])
            else:
                buf, cur_len = [], 0

    for p in paras:
        if len(p) > chunk_size:
            sentences = p.replace("\n", " ").split(". ")
            for s in sentences:
                s2 = (s + ("" if s.endswith(".") else ".")) if s else ""
                if cur_len + len(s2) + 1 > chunk_size and buf:
                    _flush_with_overlap()
                buf.append(s2)
                cur_len += len(s2) + 1
        else:
            if cur_len + len(p) + 2 > chunk_size and buf:
                _flush_with_overlap()
            buf.append(p)
            cur_len += len(p) + 2

    if buf:
        chunks.append(" ".join(buf).strip())

    return [c for c in chunks if c and len(c) > 10]

def get_st_collection_name(session_id: str) -> str:
    return f"st_pdf_{session_id}"

def get_or_create_st_collection(session_id: str):
    """Short-term per-session Chroma collection for current PDFs."""
    name = get_st_collection_name(session_id)
    try:
        return chroma_client.get_collection(
            name=name,
            embedding_function=embedding_function
        )
    except Exception:
        return chroma_client.create_collection(
            name=name,
            embedding_function=embedding_function
        )

def upsert_chunks_to_collection(collection, doc_id: str, chunks: List[str], extra_meta: Optional[Dict[str, Any]] = None):
    """Add chunks to a Chroma collection with metadata."""
    if not chunks:
        return
    docs, ids, metadatas = [], [], []
    for i, ch in enumerate(chunks):
        docs.append(ch)
        ids.append(f"{doc_id}_{i}")
        md = {"doc_id": doc_id, "chunk_index": i}
        if extra_meta:
            md.update(extra_meta)
        metadatas.append(md)
    collection.add(documents=docs, ids=ids, metadatas=metadatas)

def query_collection(collection, query: str, n: int = 5, where: Optional[Dict[str, Any]] = None) -> List[str]:
    """Return top documents text from Chroma query."""
    try:
        res = collection.query(query_texts=[query], n_results=n, where=where or {})
        return res.get("documents", [[]])[0]
    except Exception:
        return []


In [7]:
# Tools
@tool
def ingest_pdfs(paths: List[str], session_id: str) -> Dict[str, Any]:
    """
    Ingest PDFs for the current session: extract → chunk → embed to short-term store.
    Returns stats per file.
    """
    st_collection = get_or_create_st_collection(session_id)
    stats = []
    for p in paths:
        text = extract_text_from_pdf(p)
        chunks = simple_chunk_text(text)
        doc_id = f"{session_id}::{os.path.basename(p)}::{uuid.uuid4().hex[:8]}"
        upsert_chunks_to_collection(
            st_collection,
            doc_id=doc_id,
            chunks=chunks,
            extra_meta={"session_id": session_id, "file": os.path.basename(p)}
        )
        stats.append({"file": os.path.basename(p), "chunks": len(chunks), "doc_id": doc_id})
    return {"indexed_files": stats}

@tool
def summarize_current_pdf(query: str, session_id: str, k: int = 6) -> str:
    """
    Summarize/answer from SHORT-TERM (current session PDFs).
    Returns a concise answer with [S#] citations.
    """
    st_collection = get_or_create_st_collection(session_id)
    docs = query_collection(st_collection, query, n=k, where={"session_id": session_id})
    if not docs:
        return "No session PDF content found."
    system = SystemMessage(content="You are a research assistant. Summarize accurately and cite inline with [S#] indices for provided snippets.")
    context_labelled = "\n\n".join([f"[S{i+1}] {d}" for i, d in enumerate(docs)])
    user = HumanMessage(content=f"Use the snippets below to answer: {query}\n\nSnippets:\n{context_labelled}")
    out = ChatGroq(model=LLM_MODEL_NAME, groq_api_key=groq_api_key).invoke([system, user])
    return out.content

@tool
def keyword_extract(text: str, top_k: int = 10) -> List[str]:
    """
    Extract up to top_k key terms/phrases from a passage.
    """
    prompt = f"Extract up to {top_k} key terms or short phrases from the text below. Return as a bullet list.\n\nTEXT:\n{text}"
    out = ChatGroq(model=LLM_MODEL_NAME, groq_api_key=groq_api_key).invoke([HumanMessage(content=prompt)])
    lines = [ln.strip("-• ").strip() for ln in out.content.splitlines() if ln.strip()]
    uniq = []
    for l in lines:
        if l and l not in uniq:
            uniq.append(l)
    return uniq[:top_k]


@tool
def save_insight(note: str, tags: Optional[List[str]] = None, source: Optional[str] = None) -> str:
    """
    Save an insight/summary/FAQ to LONG-TERM memory (persistent).
    """
    insight_id = f"ins_{uuid.uuid4().hex[:10]}"
    tags_str = ", ".join(tags) if tags else ""
    meta = {"type": "insight", "tags": tags_str, "source": source or ""}
    lt_collection.add(documents=[note], ids=[insight_id], metadatas=[meta])
    return f"Saved long-term insight: {insight_id}"



@tool
def ask_with_memory(query: str, session_id: str, k_st: int = 5, k_lt: int = 5) -> str:
    """
    Answer a question by blending SHORT-TERM (current PDFs) + LONG-TERM (insights) contexts.
    Prefers ST evidence when conflicting. Cites with [ST#]/[LT#].
    """
    st_collection = get_or_create_st_collection(session_id)
    st_docs = query_collection(st_collection, query, n=k_st, where={"session_id": session_id}) or []
    lt_docs = query_collection(lt_collection, query, n=k_lt) or []

    if not (st_docs or lt_docs):
        return "I have no relevant context in short-term or long-term memory."

    labelled = []
    idx = 1
    for d in st_docs:
        labelled.append((f"[ST{idx}]", d))
        idx += 1
    for d in lt_docs:
        labelled.append((f"[LT{idx}]", d))
        idx += 1

    context_blob = "\n\n".join([f"{tag} {text}" for tag, text in labelled])
    prompt = [
        SystemMessage(content="You are a precise research assistant. Use ONLY the provided context unless the question is general knowledge."),
        HumanMessage(content=f"Question: {query}\n\nContext:\n{context_blob}\n\nInstructions:\n"
                             f"- Prefer short-term (ST#) evidence when it conflicts with long-term (LT#).\n"
                             f"- Cite sources inline with their tag numbers.\n"
                             f"- If insufficient context, say so and ask for a PDF or clarification.")
    ]
    out = ChatGroq(model=LLM_MODEL_NAME, groq_api_key=groq_api_key).invoke(prompt)
    return out.content

@tool
def multiply(a: int, b: int) -> int:
    """This function is responsible for multiplication"""
    return a * b


In [9]:
# BONUS 1. PDF Summaries - Long-Term

def _gather_session_docs(session_id: str):
    """Return (st_collection, files -> {doc_ids, count}, total_chunks)."""
    st_collection = get_or_create_st_collection(session_id)
    try:
        payload = st_collection.get(where={"session_id": session_id}, include=["metadatas", "documents", "ids"])
    except Exception:
        return st_collection, {}, 0

    files_map: Dict[str, Dict[str, Any]] = {}
    total_chunks = 0
    for md, _doc, _id in zip(payload.get("metadatas", []), payload.get("documents", []), payload.get("ids", [])):
        if not md:
            continue
        f = md.get("file", "unknown")
        files_map.setdefault(f, {"doc_ids": set(), "count": 0})
        files_map[f]["doc_ids"].add(md.get("doc_id", ""))
        files_map[f]["count"] += 1
        total_chunks += 1

    for f in files_map:
        files_map[f]["doc_ids"] = list(files_map[f]["doc_ids"])
    return st_collection, files_map, total_chunks

def _summarize_text_blocks(blocks: List[str], title: str) -> str:
    """Map-reduce style summary for many chunks."""
    map_msgs = [
        SystemMessage(content="You are a careful scientific summarizer. Extract key points faithfully."),
        HumanMessage(content=f"Summarize the following {len(blocks)} passages into bullet points. Be concise, no speculation.\n\n" +
                             "\n\n".join([f"[{i+1}] {b}" for i, b in enumerate(blocks[:12])]))
    ]
    mapped = ChatGroq(model=LLM_MODEL_NAME, groq_api_key=groq_api_key).invoke(map_msgs).content

    reduce_msgs = [
        SystemMessage(content="You write tight research summaries."),
        HumanMessage(content=f"Create a 3–5 sentence abstract and 5 bullets from these notes about '{title}':\n\n{mapped}")]
    return ChatGroq(model=LLM_MODEL_NAME, groq_api_key=groq_api_key).invoke(reduce_msgs).content

@tool
def summarize_pdfs_to_longterm(session_id: str, files: Optional[List[str]] = None, k_per_file: int = 40) -> str:
    """
    Create and store long-term summaries for PDFs ingested in this session.
    - If `files` is None, summarize all files in the session.
    - Stores each summary in LT memory with tags=['pdf-summary', <file>].
    """
    st_collection, files_map, _ = _gather_session_docs(session_id)
    if not files_map:
        return "No session PDFs found to summarize."

    target_files = files or list(files_map.keys())
    saved = []

    for file_name in target_files:

        docs = query_collection(
            st_collection,
            f"Main ideas and contributions of {file_name}",
            n=k_per_file,
            where={"session_id": session_id, "file": file_name}
        )
        if not docs:
            payload = st_collection.get(where={"session_id": session_id, "file": file_name}, include=["documents"])
            docs = (payload.get("documents") or [])[:k_per_file]

        if not docs:
            continue

        summary = _summarize_text_blocks(docs, title=file_name)
        insight_id = f"ins_{uuid.uuid4().hex[:10]}"

        lt_collection.add(
            documents=[summary],
            ids=[insight_id],
            metadatas=[{"type": "pdf_summary", "tags": f"pdf-summary,{file_name}", "source": file_name}]
        )
        saved.append({"file": file_name, "insight_id": insight_id})

    if not saved:
        return "No summaries created."
    return "Saved PDF summaries to long-term: " + ", ".join([f"{s['file']}→{s['insight_id']}" for s in saved])


# BONUS 2. Multi-Document Cross-Referencing

def _per_file_topk(st_collection, session_id: str, query: str, k_per_file: int = 3) -> Dict[str, List[str]]:
    """Return {file: [chunks]} with top-k matches per file."""
    _, files_map, _ = _gather_session_docs(session_id)
    result: Dict[str, List[str]] = {}
    for file_name in files_map.keys():
        docs = query_collection(
            st_collection, query, n=k_per_file,
            where={"session_id": session_id, "file": file_name}
        )
        if docs:
            result[file_name] = docs
    return result

@tool
def cross_reference(query: str, session_id: str, k_per_file: int = 3, k_lt: int = 5) -> str:
    """
    Compare evidence across multiple PDFs (ST) and augment with LT insights.
    Cites as [DOC:<file>#i] and [LT#j]. Prefer DOC when conflicting.
    """
    st_collection = get_or_create_st_collection(session_id)
    per_file = _per_file_topk(st_collection, session_id, query, k_per_file=k_per_file)
    lt_docs = query_collection(lt_collection, query, n=k_lt) or []

    if not per_file and not lt_docs:
        return "No relevant evidence found in session PDFs or long-term memory."

    tagged_blocks = []
    for f, blocks in per_file.items():
        for i, b in enumerate(blocks, start=1):
            tagged_blocks.append((f"[DOC:{f}#{i}]", b))
    for j, b in enumerate(lt_docs, start=1):
        tagged_blocks.append((f"[LT{j}]", b))

    context = "\n\n".join([f"{tag} {txt}" for tag, txt in tagged_blocks])
    prompt = [
        SystemMessage(content="Compare and contrast evidence across documents. Be precise and cite tags."),
        HumanMessage(content=f"Query: {query}\n\nEvidence:\n{context}\n\n"
                             f"Instructions:\n- Note agreements/disagreements across PDFs.\n"
                             f"- Prefer PDF (DOC) evidence over LT when conflicting.\n"
                             f"- Use clear, short paragraphs and inline citations.")]
    out = ChatGroq(model=LLM_MODEL_NAME, groq_api_key=groq_api_key).invoke(prompt)
    return out.content


# BONUS 3. Memory & Retrieval Visualization

def _count_lt() -> int:
    try:
        payload = lt_collection.get(include=[])
        return len(payload.get("ids", []))
    except Exception:
        return 0

@tool
def memory_dashboard(session_id: str, last_query: Optional[str] = None) -> Dict[str, Any]:
    """
    Return a JSON dashboard of memory usage and (optional) retrieval trace.
    """
    st_collection, files_map, total_chunks = _gather_session_docs(session_id)
    session_files = [{"file": f, "chunks": info["count"]} for f, info in files_map.items()]
    lt_count = _count_lt()

    dash: Dict[str, Any] = {
        "session": {"files": sorted(session_files, key=lambda x: x["file"]), "total_chunks": total_chunks},
        "long_term": {"total_items": lt_count}
    }

    if last_query:
        per_file = _per_file_topk(st_collection, session_id, last_query, k_per_file=5)
        lt_docs = query_collection(lt_collection, last_query, n=5) or []
        dash["retrieval"] = {
            "query": last_query,
            "per_file_hits": {f: len(v) for f, v in per_file.items()},
            "lt_hits": len(lt_docs)
        }

    return dash

@tool
def memory_dashboard_markdown(session_id: str, last_query: Optional[str] = None) -> str:
    """
    Pretty Markdown view of the memory dashboard for quick visualization in-chat.
    """
    data = memory_dashboard(session_id, last_query)
    ses = data["session"]; lt = data["long_term"]
    lines = ["# Memory Dashboard",
             f"**Session:** `{session_id}`",
             "## Short-Term (Session PDFs)",
             "",
             "| File | Chunks |",
             "|---|---|"]
    for item in ses["files"]:
        lines.append(f"| {item['file']} | {item['chunks']} |")
    lines += ["", f"**Total session chunks:** {ses['total_chunks']}", "",
              "## Long-Term Memory",
              f"**Total items:** {lt['total_items']}"]

    if "retrieval" in data:
        r = data["retrieval"]
        lines += ["", "## Retrieval Trace",
                  f"**Query:** {r['query']}",
                  "", "| Source | Hits |", "|---|---|"]
        for f, k in r["per_file_hits"].items():
            lines.append(f"| {f} | {k} |")
        lines.append(f"| Long-Term | {r['lt_hits']} |")

    return "\n".join(lines)


# Session End

@tool
def end_session(session_id: str) -> str:
    """
    Delete the short-term (session) vector store to forget context.
    """
    name = get_st_collection_name(session_id)
    try:
        chroma_client.delete_collection(name)
    except InvalidCollectionException:
        pass
    return f"Short-term memory for session '{session_id}' cleared."

In [10]:
# State + Agent Node

class State(TypedDict):
    messages: Annotated[List[Any], add_messages]
    session_id: str                   # short-term session scope
    conversation_id: str              # long-term user/conv scope
    uploaded_files: List[str]         # paths for PDFs in this session

# Tools registry
TOOLS = [
    ingest_pdfs,
    summarize_current_pdf,
    keyword_extract,
    save_insight,
    ask_with_memory,
    multiply,
    summarize_pdfs_to_longterm,
    cross_reference,
    memory_dashboard,
    memory_dashboard_markdown,
    end_session
]

llm_with_tools = llm.bind_tools(TOOLS)

def _blend_context_with_retrieval(state: State) -> str:
    """
    Retrieve best-effort context for last user message from ST + LT (for prompting).
    Prefers including both, but final answers should still cite via tools.
    """
    session_id = state.get("session_id", "")
    last_user = ""
    for m in reversed(state["messages"]):
        if isinstance(m, HumanMessage) or (isinstance(m, tuple) and m[0] == "user"):
            last_user = m.content if hasattr(m, "content") else m[1]
            break
    if not last_user:
        return ""

    st_docs: List[str] = []
    if session_id:
        st_collection = get_or_create_st_collection(session_id)
        st_docs = query_collection(st_collection, last_user, n=4, where={"session_id": session_id})

    lt_docs = query_collection(lt_collection, last_user, n=4)

    st_blob = "\n\n".join([f"[ST{i+1}] {d}" for i, d in enumerate(st_docs)]) if st_docs else ""
    lt_blob = "\n\n".join([f"[LT{i+1}] {d}" for i, d in enumerate(lt_docs)]) if lt_docs else ""

    context = "\n\n".join([b for b in [st_blob, lt_blob] if b])
    return context

def agent_node(state: State):
    """
    Main LLM node:
    - Retrieves context (ST + LT) to prime the LLM
    - Lets the LLM decide whether to call tools via tools_condition.
    """
    context = _blend_context_with_retrieval(state)
    if context:
        messages_for_llm = [
            SystemMessage(content="You are a rigorous research assistant. Use the provided context when relevant."),
            HumanMessage(content=f"Context:\n{context}")
        ] + state["messages"]
    else:
        messages_for_llm = state["messages"]

    return {"messages": [llm_with_tools.invoke(messages_for_llm)]}

In [11]:
# Graph Assembly
memory = MemorySaver()

builder = StateGraph(State)
builder.add_node("agent", agent_node)
builder.add_node("tools", ToolNode(TOOLS))
builder.add_edge(START, "agent")
builder.add_conditional_edges("agent", tools_condition)
builder.add_edge("tools", END)

graph = builder.compile(checkpointer=memory)

In [12]:
#  Evaluation
# Session A: Ingest PDFs and ask questions (SHORT-TERM in action)
session_a = "session_A_001"
cfg_a = {"configurable": {"thread_id": session_a}}

pdf_paths = ["/content/SA_MLP__Accurate_and_Interpretable_Breast_Cancer_Detection_with_Feature_Selection_and_Explainable_AI.pdf"]
r0 = graph.invoke({
  "messages": [HumanMessage(content=f"Please ingest these PDFs: {pdf_paths}. Use session_id='{session_a}'.")],
  "session_id": session_a,
  "conversation_id": "user_abc",
  "uploaded_files": pdf_paths
}, cfg_a)
print("\n[Agent] Ingestion response:\n", r0["messages"][-1].content)


# Ask a session-specific question
q1 = "What are the main contributions listed in the paper?"
r1 = graph.invoke({
  "messages": [HumanMessage(content=f"Question about the PDF (session {session_a}): {q1}")],
  "session_id": session_a,
  "conversation_id": "user_abc",
  "uploaded_files": pdf_paths
}, cfg_a)
print("\n[Agent] Q1 answer:\n", r1["messages"][-1].content)


# Save an insight to LONG-TERM memory
insight_text = "The paper introduces the SA based Algorithm and Grid search framework."
r2 = graph.invoke({
  "messages": [HumanMessage(content=f"Save this as a long-term insight: {insight_text}")],
  "session_id": session_a,
  "conversation_id": "user_abc"
}, cfg_a)
print("\n[Agent] Save insight response:\n", r2["messages"][-1].content)



/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 104MiB/s]



[Agent] Ingestion response:
 {"indexed_files": [{"file": "SA_MLP__Accurate_and_Interpretable_Breast_Cancer_Detection_with_Feature_Selection_and_Explainable_AI.pdf", "chunks": 31, "doc_id": "session_A_001::SA_MLP__Accurate_and_Interpretable_Breast_Cancer_Detection_with_Feature_Selection_and_Explainable_AI.pdf::df6af01b"}]}

[Agent] Q1 answer:
 **Main Contributions**

1. **Optimized learning pipeline** – The study introduces a two‑stage optimization: simulated‑annealing–based feature selection followed by grid‑search hyperparameter tuning. This pipeline delivers a robust model (SA‑MLP) that achieves the highest reported accuracy on the Wisconsin Breast Cancer Dataset (WBCD) at 98.60 % [S1].

2. **Systematic data preparation and model tuning** – A comprehensive methodology is presented that includes careful data cleaning, feature selection, and exhaustive hyperparameter tuning. This process not only maximizes predictive performance but also enhances interpretability for clinical decision

In [13]:
# Summarize all PDFs in the session to LONG-TERM memory (BONUS 1)
r2b = graph.invoke({
  "messages": [HumanMessage(content=f"summarize_pdfs_to_longterm(session_id='{session_a}')")],
  "session_id": session_a,
  "conversation_id": "user_abc"
}, cfg_a)
print("\n[Agent] PDF summaries -> LT:\n", r2b["messages"][-1].content)

# Cross-reference a query across multiple PDFs + LT (BONUS 2)
r2c = graph.invoke({
  "messages": [HumanMessage(content=f"cross_reference(query='How do they handle positional encoding?', session_id='{session_a}', k_per_file=3, k_lt=3)")],
  "session_id": session_a,
  "conversation_id": "user_abc"
}, cfg_a)
print("\n[Agent] Cross-reference answer:\n", r2c["messages"][-1].content)

#Visualize memory usage and retrieval trace (BONUS 3)
r2d = graph.invoke({
  "messages": [HumanMessage(content=f"memory_dashboard_markdown(session_id='{session_a}', last_query='Transformer scalability')")],
  "session_id": session_a,
  "conversation_id": "user_abc"
}, cfg_a)
print("\n[Agent] Memory Dashboard (Markdown):\n", r2d["messages"][-1].content)


[Agent] PDF summaries -> LT:
 No session PDFs found to summarize.

[Agent] Cross-reference answer:
 No relevant evidence found in session PDFs or long-term memory.

[Agent] Memory Dashboard (Markdown):
 Error: AttributeError("'str' object has no attribute 'parent_run_id'")
 Please fix your mistakes.


/tmp/ipython-input-2019503163.py:169: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  data = memory_dashboard(session_id, last_query)


In [14]:
#Session B: New session, reuse LONG-TERM memory
session_b = "session_B_009"
cfg_b = {"configurable": {"thread_id": session_b}}

q2 = "Remind me what that paper contributed overall."
r3 = graph.invoke({
  "messages": [HumanMessage(content=f"In a new session ({session_b}), {q2}")],
  "session_id": session_b,
  "conversation_id": "user_abc"
}, cfg_b)
print("\n[Agent] Q2 (new session, uses LT):\n", r3["messages"][-1].content)

#Explicit blended query tool (ST + LT)
r4 = graph.invoke({
  "messages": [HumanMessage(content=f"ask_with_memory(query='{q1}', session_id='{session_a}', k_st=5, k_lt=5)")],
  "session_id": session_a,
  "conversation_id": "user_abc"
}, cfg_a)
print("\n[Agent] ask_with_memory result:\n", r4["messages"][-1].content)

#End Session
r_end = graph.invoke({
  "messages": [HumanMessage(content=f"end_session(session_id='{session_a}')")],
  "session_id": session_a,
  "conversation_id": "user_abc"
}, cfg_a)
print("\n[Agent] End session A:\n", r_end["messages"][-1].content)



[Agent] Q2 (new session, uses LT):
 I’m happy to help, but I need to know which paper you’re referring to. Could you let me know the title, authors, or any key details so I can pull up the right information?

[Agent] ask_with_memory result:
 **Main contributions of the paper**

1. **Proposed a novel SA‑MLP model** for breast‑cancer diagnosis that integrates *Simulated Annealing* (SA) for feature selection with a standard Multi‑Layer Perceptron (MLP) classifier.  
   *SA* reduces the dimensionality of the Wisconsin Breast Cancer Dataset (WBCD) while preserving discriminative information, and the resulting SA‑MLP achieves an accuracy of **98.60 %**【ST5】.  

2. **Comprehensive hyper‑parameter optimisation** using Grid Search.  The best‑performing settings for each algorithm (MLP, XGBoost, SVC, Random Forest) are reported and used to evaluate the models under 5‑fold cross‑validation【ST1】【ST4】.  

3. **Extensive evaluation and comparison** with state‑of‑the‑art methods.  The paper presents